In [47]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [48]:
vic_data = pd.read_csv('../kennedy/victoria.csv')

In [3]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

In [4]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [5]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['charge_forecast'] = 0
    df['discharge_forecast'] = 0
    
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    df = df.drop([0], axis=0) 
    
    return df

In [69]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
            
    return df

In [51]:
def calculate(df):
    
    for i in list(df.index):
        price = df.at[i, PRICE]
        
        # update opening capacity
        if (i != 1):
            df.at[i, 'opening'] = df.at[i-1, 'closing']
            
        opening_cap = df.at[i, 'opening']


        # finding raw_power
        if (df.at[i,'charge_forecast'] == 1):
            df.at[i,'raw_power'] = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)

        if (df.at[i,'discharge_forecast'] == 1):
            df.at[i,'raw_power'] = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
        rawPower = df.at[i, "raw_power"]


        # finding market_dispatch 
        if (rawPower < 0):
            df.at[i,'dispatch'] = (rawPower/2)

        else:
            # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
            df.at[i,'dispatch'] = (rawPower/2 * EFFICIENCY)
            
        dispatch = df.at[i, "dispatch"]


        # finding closing_capacity   
        if (dispatch < 0):
            thecondition = opening_cap - (dispatch * EFFICIENCY)
        else:
            thecondition = opening_cap - (dispatch * (100/(EFFICIENCY*100)))

        df.at[i,'closing'] = round(max(0, min(thecondition, BATTERY_CAPACITY)),0)



        #finding revenue        
        if (dispatch < 0):
            df.at[i,'revenue'] = round(dispatch * price * (1/MARGINAL_LOSS_FACTOR),0)
        else:
            df.at[i,'revenue'] = round(dispatch * price * MARGINAL_LOSS_FACTOR,0)
    
    return df

In [61]:
def run_all(ori_df):
    
    start = time.time()
    
    df2 = create_df(ori_df)
    
    df3 = algorithm3(df2)
    
    df = calculate(df3)
            
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [70]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.3
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 121879439
Total days in the dataset: 1322.0
Revenue per day: 92193.22163388805
Time Complexity for running the entire Algorithm 3: 22.599504947662354s


In [ ]:
# Current highest revenue: 92193

############################################# This is a line breaker #################################################

# DELIVERABLE 3

In [64]:
del3 = vic_price[(vic_price[TIME] > "2020-07-17 00:00:00") & (vic_price[TIME] < "2020-07-18 00:00:30")]
print("Total revenue in the dataset:", del3["revenue"].sum())
print("Total days in the dataset:", len(del3)/48)
print("Revenue per day:", del3["revenue"].sum() / (len(del3)/48))
del3

Total revenue in the dataset: 49239
Total days in the dataset: 1.0
Revenue per day: 49239.0


,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
44545,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
44546,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
44547,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
44548,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
44549,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
44550,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
44551,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
44552,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
44553,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0
44554,2020-07-17 05:00:00,67.32,-300,-150,-10190,270,405,1,0
